# 🎯 AI Clinic - Waiting Time Prediction Model Evaluation
## Machine Learning Model Performance Analysis

**Model**: Random Forest Regressor  
**Task**: Predict patient waiting time in minutes  
**Features**: 7 input features (hour, day, queue position, etc.)

In [ ]:
# Setup Django environment
import os
import sys
import django

sys.path.append('c:/Users/VITUS/AiClinicNew/ClinicProject')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'clinic_token_system.settings')
django.setup()

print("✅ Django environment loaded successfully!")

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix
from api.waiting_time_predictor import waiting_time_predictor
from api.models import Token
from datetime import timedelta
from django.utils import timezone

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported successfully!")

## 📊 Step 1: Load Training Data

In [ ]:
# Prepare training data
X, y = waiting_time_predictor.prepare_training_data(use_all_data=True)

print(f"📊 Training Data Shape:")
print(f"   Features (X): {X.shape}")
print(f"   Target (y): {y.shape}")
print(f"\n📈 Sample Statistics:")
print(f"   Mean waiting time: {np.mean(y):.2f} minutes")
print(f"   Std deviation: {np.std(y):.2f} minutes")
print(f"   Min: {np.min(y):.2f} minutes")
print(f"   Max: {np.max(y):.2f} minutes")

## 🎓 Step 2: Train Model & Get Predictions

In [ ]:
from sklearn.model_selection import train_test_split

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
print("🔄 Training model...")
success = waiting_time_predictor.train_model()

if success:
    print("✅ Model trained successfully!")
    
    # Load model and make predictions
    waiting_time_predictor.load_model()
    X_test_scaled = waiting_time_predictor.scaler.transform(X_test)
    y_pred = waiting_time_predictor.model.predict(X_test_scaled)
    
    print(f"\n📊 Predictions generated: {len(y_pred)} samples")
else:
    print("❌ Model training failed!")

## 📈 Step 3: Regression Metrics (For Continuous Prediction)

In [ ]:
# Calculate regression metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Calculate MAPE (Mean Absolute Percentage Error)
mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

print("="*60)
print("🎯 REGRESSION METRICS (Continuous Prediction)")
print("="*60)
print(f"\n📊 Mean Absolute Error (MAE):     {mae:.2f} minutes")
print(f"   → On average, predictions are off by {mae:.2f} minutes")
print(f"\n📊 Root Mean Squared Error (RMSE): {rmse:.2f} minutes")
print(f"   → Penalizes larger errors more heavily")
print(f"\n📊 R² Score (Coefficient of Determination): {r2:.4f}")
print(f"   → Model explains {r2*100:.2f}% of variance in waiting times")
print(f"\n📊 Mean Absolute Percentage Error (MAPE): {mape:.2f}%")
print(f"   → Average prediction error is {mape:.2f}% of actual value")
print("\n" + "="*60)

## 🎯 Step 4: Classification Metrics (Accuracy & F1 Score)

**Note**: Since this is a regression problem, we convert it to classification by binning predictions into categories:
- **Short Wait**: 0-15 minutes
- **Medium Wait**: 16-30 minutes
- **Long Wait**: 31+ minutes

In [ ]:
# Convert continuous predictions to categories
def categorize_wait_time(minutes):
    if minutes <= 15:
        return 'Short (0-15 min)'
    elif minutes <= 30:
        return 'Medium (16-30 min)'
    else:
        return 'Long (31+ min)'

# Categorize actual and predicted values
y_test_cat = [categorize_wait_time(y) for y in y_test]
y_pred_cat = [categorize_wait_time(y) for y in y_pred]

# Calculate classification metrics
accuracy = accuracy_score(y_test_cat, y_pred_cat)
f1 = f1_score(y_test_cat, y_pred_cat, average='weighted')

print("="*60)
print("🎯 CLASSIFICATION METRICS (Categorical Prediction)")
print("="*60)
print(f"\n✅ Accuracy Score: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   → Model correctly predicts wait time category {accuracy*100:.2f}% of the time")
print(f"\n✅ F1 Score (Weighted): {f1:.4f}")
print(f"   → Harmonic mean of precision and recall: {f1*100:.2f}%")
print("\n" + "="*60)

## 📊 Step 5: Detailed Classification Report

In [ ]:
# Generate classification report
print("\n📋 DETAILED CLASSIFICATION REPORT:")
print("="*60)
print(classification_report(y_test_cat, y_pred_cat))

# Confusion Matrix
cm = confusion_matrix(y_test_cat, y_pred_cat)
print("\n📊 CONFUSION MATRIX:")
print(cm)

## 📈 Step 6: Visualizations

In [ ]:
# 1. Actual vs Predicted Scatter Plot
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred, alpha=0.5)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('Actual Waiting Time (minutes)', fontsize=12)
plt.ylabel('Predicted Waiting Time (minutes)', fontsize=12)
plt.title('Actual vs Predicted Waiting Times', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("✅ Scatter plot shows how close predictions are to actual values")
print("   Points closer to red line = better predictions")

In [ ]:
# 2. Prediction Error Distribution
errors = y_test - y_pred

plt.figure(figsize=(10, 6))
plt.hist(errors, bins=30, edgecolor='black', alpha=0.7)
plt.axvline(x=0, color='r', linestyle='--', linewidth=2, label='Perfect Prediction')
plt.xlabel('Prediction Error (minutes)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Distribution of Prediction Errors', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"✅ Error distribution centered around {np.mean(errors):.2f} minutes")
print(f"   Most predictions within ±{np.std(errors):.2f} minutes")

In [ ]:
# 3. Confusion Matrix Heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Short', 'Medium', 'Long'],
            yticklabels=['Short', 'Medium', 'Long'])
plt.xlabel('Predicted Category', fontsize=12)
plt.ylabel('Actual Category', fontsize=12)
plt.title('Confusion Matrix - Wait Time Categories', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("✅ Diagonal values show correct predictions")
print("   Off-diagonal values show misclassifications")

In [ ]:
# 4. Metrics Summary Bar Chart
metrics = {
    'Accuracy': accuracy * 100,
    'F1 Score': f1 * 100,
    'R² Score': r2 * 100
}

plt.figure(figsize=(10, 6))
bars = plt.bar(metrics.keys(), metrics.values(), color=['#2ecc71', '#3498db', '#e74c3c'])
plt.ylabel('Score (%)', fontsize=12)
plt.title('Model Performance Metrics', fontsize=14, fontweight='bold')
plt.ylim(0, 100)
plt.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.1f}%',
             ha='center', va='bottom', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("✅ All metrics above 80% indicate excellent model performance")

## 📊 Step 7: Final Summary Report

In [ ]:
print("\n" + "="*70)
print("🏆 FINAL MODEL EVALUATION SUMMARY")
print("="*70)
print(f"\n📊 Dataset Information:")
print(f"   Total Samples: {len(X)}")
print(f"   Training Samples: {len(X_train)}")
print(f"   Testing Samples: {len(X_test)}")
print(f"   Features: {X.shape[1]}")

print(f"\n🎯 Regression Metrics (Continuous Prediction):")
print(f"   MAE:  {mae:.2f} minutes")
print(f"   RMSE: {rmse:.2f} minutes")
print(f"   R²:   {r2:.4f} ({r2*100:.2f}%)")
print(f"   MAPE: {mape:.2f}%")

print(f"\n🎯 Classification Metrics (Categorical Prediction):")
print(f"   Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print(f"   F1 Score: {f1:.4f} ({f1*100:.2f}%)")

print(f"\n✅ Model Performance Rating:")
if accuracy >= 0.85 and r2 >= 0.80:
    print("   🌟 EXCELLENT - Model is production-ready!")
elif accuracy >= 0.75 and r2 >= 0.70:
    print("   ✅ GOOD - Model performs well")
elif accuracy >= 0.65 and r2 >= 0.60:
    print("   ⚠️  FAIR - Model needs improvement")
else:
    print("   ❌ POOR - Model needs retraining")

print("\n" + "="*70)
print("✅ Evaluation Complete!")
print("="*70)